In [33]:
import pandas as pd
import os
import sqlite3
import numpy as np

import geopandas as gpd
from lets_plot.geo_data import *
from numerize import numerize as nz

from lets_plot import * # This imports all of ggplot2's functions
LetsPlot.setup_html() 

In [34]:
DATA_FOLDER_P = os.path.join('../data')
conn = sqlite3.connect(os.path.join(DATA_FOLDER_P, './home_prices.db'))


In [35]:
zip_df = pd.read_sql_query("SELECT * FROM zip_info", conn)
zillow_df = pd.read_sql_query("SELECT * FROM zillow", conn)
display(zillow_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              766 non-null    int64  
 1   price           766 non-null    float64
 2   address         766 non-null    object 
 3   addressZipcode  766 non-null    object 
 4   addressState    766 non-null    object 
 5   beds            766 non-null    int64  
 6   baths           766 non-null    float64
 7   area            766 non-null    float64
 8   zestimate       588 non-null    float64
 9   brokerName      721 non-null    object 
 10  town            766 non-null    object 
dtypes: float64(4), int64(2), object(5)
memory usage: 66.0+ KB


None

In [36]:
agg_funcs = {
    'price': ['mean', 'median', 
                 ('25th', lambda x: np.percentile(x, 25)),  
                 ('75th', lambda x: np.percentile(x, 75)),  
                 'min', 'max', 'count']
}
#using code from midterm
price_zip = zillow_df.groupby('addressZipcode').agg(agg_funcs)
#combining the two header lines onto one line
price_zip.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in price_zip.columns]
price_zip = price_zip.reset_index()
# display(price_zip.head())

In [37]:
p = ggplot(price_zip) + \
    geom_boxplot(aes(x='addressZipcode', ymin='price_min', lower='price_25th', middle='price_median', upper='price_75th', ymax='price_max'), stat='identity') +\
    scale_y_log10() 
    

p.show()

In [41]:
shapefile_path = '../data/ZIPCODES_NT_POLY.sbn'
gdf = gpd.read_file(shapefile_path, SHAPE_RESTORE_SHX='YES')
# gdf = gpd.read_file(shapefile_path)

DataSourceError: '../data/ZIPCODES_NT_POLY.sbn' not recognized as a supported file format. It might help to specify the correct driver explicitly by prefixing the file path with '<DRIVER>:', e.g. 'CSV:path'.